# Step 08: Control Totals

This notebook executes control totals SQL scripts and compares Contract Import File (3b) vs RMS EDM (3d) results.

**Tasks:**
- Load configuration metadata (date_value, cycle_type)
- Validate Workspace EDM
- Execute 3b_Control_Totals_Contract_Import_File_Tables.sql
- Execute 3d_RMS_EDM_Control_Totals.sql
- Compare 3b vs 3d results and display differences
- Execute 3e_GeocodingSummary.sql and validate GeoHaz thresholds
- Compare 3d vs 3e results for base portfolios

## 1) Setup

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from helpers.notebook_setup import initialize_notebook_context
from helpers import ux
from helpers.database import execute_query
from helpers.sqlserver import execute_query_from_file, sql_file_exists
from helpers.constants import WORKSPACE_PATH
from helpers.configuration import read_configuration, get_base_portfolios
from helpers.irp_integration import IRPClient
from helpers.control_totals import (
    compare_3b_vs_3d_pivot,
    compare_3d_vs_3e_pivot,
    validate_geohaz_thresholds,
    get_import_file_mapping_from_config,
    get_exposure_group_portname_mapping
)
from helpers.excel_export import (
    save_control_totals_3b_vs_3d_to_excel,
    save_geohaz_validation_to_excel
)
from helpers.csv_export import save_dataframes_to_csv

# Flag to track execution state
execution_failed = False
error_message = None

In [ ]:
# Initialize notebook context and step tracking
context, step = initialize_notebook_context('Step_08_Control_Totals.ipynb')

# Display context
ux.header("Control Totals Execution")
ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")
ux.success(f"✓ Step tracking initialized for '{context.step_name}'")

## 2) Load Configuration

In [ ]:
# Load configuration to get date_value and cycle_type
ux.subheader("Load Configuration")

# Query for active configuration
query = """
    SELECT c.id, c.configuration_data
    FROM irp_configuration c
    INNER JOIN irp_cycle cy ON c.cycle_id = cy.id
    WHERE cy.cycle_name = %s
      AND c.status IN ('VALID', 'ACTIVE')
    ORDER BY c.created_ts DESC
    LIMIT 1
"""

result = execute_query(query, (context.cycle_name,))

if result.empty:
    execution_failed = True
    error_message = "No valid configuration found for this cycle"
    ux.error(f"✗ {error_message}")
else:
    config_id = int(result.iloc[0]['id'])
    config_data = result.iloc[0]['configuration_data']
    metadata = config_data.get('Metadata', {})
    
    date_value = metadata.get('Current Date Value', '')
    cycle_type = metadata.get('Cycle Type', '')
    
    if not date_value or not cycle_type:
        execution_failed = True
        error_message = f"Missing required metadata: date_value={date_value}, cycle_type={cycle_type}"
        ux.error(f"✗ {error_message}")
    else:
        ux.success(f"✓ Configuration loaded")
        ux.info(f"  Configuration ID: {config_id}")
        ux.info(f"  Date Value: {date_value}")
        ux.info(f"  Cycle Type: {cycle_type}")
        step.log(f"Configuration loaded: date_value={date_value}, cycle_type={cycle_type}")

## 3) Validate Workspace EDM

In [ ]:
# Get Workspace EDM
workspace_edm_full_name = None

if execution_failed:
    ux.warning("⏭ Skipping EDM validation due to configuration error")
else:
    ux.subheader("Validate Workspace EDM")
    
    try:
        irp_client = IRPClient()
        workspace_edm = "WORKSPACE_EDM"
        workspace_edms = irp_client.edm.search_edms(filter=f"exposureName=\"{workspace_edm}\"")
        
        if len(workspace_edms) == 0:
            moody_job_id = irp_client.edm.submit_create_edm_job(edm_name=workspace_edm)
            irp_client.job.poll_risk_data_job_to_completion(moody_job_id)
            workspace_edms = irp_client.edm.search_edms(filter=f"exposureName=\"{workspace_edm}\"")
        
        workspace_edm_obj = workspace_edms[0]
        workspace_edm_full_name = workspace_edm_obj['databaseName']
        
        ux.success(f"✓ Workspace EDM validated")
        ux.info(f"  EDM Full Name: {workspace_edm_full_name}")
        step.log(f"Workspace EDM: {workspace_edm_full_name}")
        
    except Exception as e:
        execution_failed = True
        error_message = f"Failed to validate Workspace EDM: {str(e)}"
        ux.error(f"✗ {error_message}")

## 4) Execute Import File Control Totals (3b)

In [ ]:
# Execute 3b_Control_Totals_Contract_Import_File_Tables.sql
import_file_results = []
csv_3b_path = None

if execution_failed:
    ux.warning("⏭ Skipping 3b execution due to previous error")
else:
    ux.subheader("Execute Import File Control Totals (3b)")
    
    sql_file_3b = WORKSPACE_PATH / 'sql' / 'control_totals' / '3b_Control_Totals_Contract_Import_File_Tables.sql'
    
    if not sql_file_exists(sql_file_3b):
        ux.warning(f"⚠ SQL file not found: {sql_file_3b}")
        ux.info("Skipping 3b execution")
    else:
        ux.info(f"Executing: {sql_file_3b.name}")
        ux.info(f"Parameters: DATE_VALUE={date_value}, CYCLE_TYPE={cycle_type}")
        ux.info("")
        
        try:
            # Execute SQL script - returns list of DataFrames (one per SELECT)
            import_file_results = execute_query_from_file(
                sql_file_3b,
                params={'DATE_VALUE': date_value, 'CYCLE_TYPE': cycle_type},
                connection='ASSURANT',
                database='DW_EXP_MGMT_USER'
            )
            
            ux.success(f"✓ Executed 3b script: {len(import_file_results)} result set(s)")
            step.log(f"Executed 3b script: {len(import_file_results)} result sets")
            
            # Save combined results to CSV for investigation (comma-delimited)
            if import_file_results:
                notebook_dir = context.notebook_path.parent
                df_3b_combined = pd.concat(import_file_results, ignore_index=True)
                csv_3b_paths = save_dataframes_to_csv(
                    df_3b_combined,
                    f"Control_Totals_3b_{date_value}",
                    output_dir=notebook_dir,
                    delimiter=','
                )
                csv_3b_path = csv_3b_paths[0] if csv_3b_paths else None
                if csv_3b_path:
                    ux.info(f"  Raw data saved to: {csv_3b_path.name}")
            
        except Exception as e:
            ux.error(f"✗ Error executing 3b script: {str(e)}")
            step.log(f"Error executing 3b script: {str(e)}")

## 5) Execute RMS EDM Control Totals (3d)

In [ ]:
# Execute 3d_RMS_EDM_Control_Totals.sql
edm_results = []
df_3d_normalized = None
csv_3d_path = None

if execution_failed:
    ux.warning("⏭ Skipping 3d execution due to previous error")
elif workspace_edm_full_name is None:
    ux.warning("⚠ Cannot execute 3d: Workspace EDM not available")
else:
    ux.subheader("Execute RMS EDM Control Totals (3d)")

    sql_file_3d = WORKSPACE_PATH / 'sql' / 'control_totals' / '3d_RMS_EDM_Control_Totals.sql'

    if not sql_file_exists(sql_file_3d):
        ux.warning(f"⚠ SQL file not found: {sql_file_3d}")
        ux.info("Skipping 3d execution")
    else:
        ux.info(f"Executing: {sql_file_3d.name}")
        ux.info(f"Parameters: WORKSPACE_EDM={workspace_edm_full_name}, DATE_VALUE={date_value}, CYCLE_TYPE={cycle_type}")
        ux.info("")

        try:
            # Execute SQL script - returns list of DataFrames (10 result sets)
            edm_results = execute_query_from_file(
                sql_file_3d,
                params={
                    'WORKSPACE_EDM': workspace_edm_full_name,
                    'DATE_VALUE': date_value,
                    'CYCLE_TYPE': cycle_type
                },
                connection='DATABRIDGE'
            )

            ux.success(f"✓ Executed 3d script: {len(edm_results)} result set(s)")
            step.log(f"Executed 3d script: {len(edm_results)} result sets")

            # Normalize the 10 result sets into a single DataFrame (merged by PORTNAME)
            if edm_results and len(edm_results) >= 10:
                from helpers.control_totals import normalize_3d_results
                df_3d_normalized = normalize_3d_results(edm_results)

                if df_3d_normalized is not None and not df_3d_normalized.empty:
                    ux.success(f"✓ Normalized 3d results: {len(df_3d_normalized)} unique PORTNAMEs")

                    # Save normalized results to CSV for investigation (comma-delimited)
                    notebook_dir = context.notebook_path.parent
                    csv_3d_paths = save_dataframes_to_csv(
                        df_3d_normalized,
                        f"Control_Totals_3d_{date_value}",
                        output_dir=notebook_dir,
                        delimiter=','
                    )
                    csv_3d_path = csv_3d_paths[0] if csv_3d_paths else None
                    if csv_3d_path:
                        ux.info(f"  Normalized data saved to: {csv_3d_path.name}")

        except Exception as e:
            ux.error(f"✗ Error executing 3d script: {str(e)}")
            step.log(f"Error executing 3d script: {str(e)}")

## 6) Compare Import File (3b) vs RMS EDM (3d) Control Totals

Compares control totals between 3b (Contract Import File) and 3d (RMS EDM):

**Non-Flood perils** (7 attributes): PolicyCount, PolicyPremium, PolicyLimit, LocationCountDistinct, TotalReplacementValue, LocationLimit, LocationDeductible

**Flood perils** (10 attributes): Adds AttachmentPoint, PolicyDeductible, PolicySublimit

**A difference of 0 means the values match.** Detailed results are exported to Excel.

In [ ]:
# Compare 3b vs 3d control totals
comparison_results = None
all_matched = False
non_flood_summary = {}
flood_summary = {}

if execution_failed:
    ux.warning("⏭ Skipping comparison due to previous error")
elif not import_file_results or not edm_results:
    ux.warning("⚠ Cannot compare: Missing 3b or 3d results")
else:
    ux.subheader("3b vs 3d Comparison Results")
    
    try:
        # Get ExposureGroup -> Portname mapping from configuration
        exposure_group_mapping = get_exposure_group_portname_mapping(config_data)
        
        # Run comparison (pivot format - one row per PORTNAME)
        comparison_results, all_matched = compare_3b_vs_3d_pivot(
            import_file_results,
            edm_results,
            exposure_group_mapping
        )
        
        if comparison_results is not None and not comparison_results.empty:
            # Split into Flood and Non-Flood for separate summaries
            is_flood = comparison_results['PORTNAME'].str.startswith('USFL_')
            flood_results = comparison_results[is_flood]
            non_flood_results = comparison_results[~is_flood]
            
            # Non-Flood summary
            if not non_flood_results.empty:
                non_flood_total = len(non_flood_results)
                non_flood_matched = (non_flood_results['Status'] == 'MATCH').sum()
                non_flood_mismatched = non_flood_total - non_flood_matched
                non_flood_summary = {
                    'total': non_flood_total,
                    'matched': int(non_flood_matched),
                    'mismatched': int(non_flood_mismatched)
                }
                
                ux.info("Non-Flood Perils:")
                if non_flood_mismatched == 0:
                    ux.success(f"  ✓ All {non_flood_total} exposure groups match")
                else:
                    ux.warning(f"  ⚠ {non_flood_mismatched} of {non_flood_total} exposure groups have mismatches")
            
            # Flood summary
            if not flood_results.empty:
                flood_total = len(flood_results)
                flood_matched = (flood_results['Status'] == 'MATCH').sum()
                flood_mismatched = flood_total - flood_matched
                flood_summary = {
                    'total': flood_total,
                    'matched': int(flood_matched),
                    'mismatched': int(flood_mismatched)
                }
                
                ux.info("Flood Perils (USFL_*):")
                if flood_mismatched == 0:
                    ux.success(f"  ✓ All {flood_total} exposure groups match")
                else:
                    ux.warning(f"  ⚠ {flood_mismatched} of {flood_total} exposure groups have mismatches")
            
            # Log to step
            total_groups = len(comparison_results)
            matched_groups = (comparison_results['Status'] == 'MATCH').sum()
            step.log(f"3b vs 3d comparison complete: {matched_groups}/{total_groups} groups matched")
        else:
            ux.warning("No comparison results generated")
            
    except Exception as e:
        ux.error(f"✗ Error comparing control totals: {str(e)}")
        step.log(f"Error comparing control totals: {str(e)}")

### Export Comparison Results to Excel

Results will be exported after 3d vs 3e comparison is complete.

In [ ]:
# Export will happen after 3d vs 3e comparison
# See "Export Control Totals to Excel" section below
ux.info("Comparison export deferred until after 3d vs 3e comparison")

## 7) Execute GeoHaz Control Totals (3e)

In [ ]:
# Execute GeoHaz Control Totals Query
geocoding_results = None
csv_3e_path = None

if execution_failed:
    ux.warning("⏭ Skipping 3e execution due to previous error")
elif workspace_edm_full_name is None:
    ux.warning("⚠ Cannot execute 3e: Workspace EDM not available")
else:
    ux.subheader("Execute GeoHaz Control Totals (3e)")
    
    sql_file_3e = WORKSPACE_PATH / 'sql' / 'control_totals' / '3e_GeocodingSummary.sql'
    
    if not sql_file_exists(sql_file_3e):
        ux.warning(f"⚠ SQL file not found: {sql_file_3e}")
        ux.info("Skipping 3e execution")
    else:
        ux.info(f"Executing: {sql_file_3e.name}")
        ux.info(f"Parameters: WORKSPACE_EDM={workspace_edm_full_name}, DATE_VALUE={date_value}, CYCLE_TYPE={cycle_type}")
        ux.info("")
        
        try:
            result = execute_query_from_file(
                sql_file_3e,
                params={
                    'WORKSPACE_EDM': workspace_edm_full_name,
                    'CYCLE_TYPE': cycle_type,
                    'DATE_VALUE': date_value
                },
                connection='DATABRIDGE'
            )
            
            # Extract geocoding results (first result set)
            geocoding_results = result[0] if isinstance(result, list) else result
            
            ux.success(f"✓ Executed 3e script: {len(geocoding_results)} geocoding records retrieved")
            step.log(f"Executed 3e script: {len(geocoding_results)} geocoding records")
            
            # Save results to CSV for investigation (comma-delimited)
            if geocoding_results is not None and not geocoding_results.empty:
                notebook_dir = context.notebook_path.parent
                csv_3e_paths = save_dataframes_to_csv(
                    geocoding_results,
                    f"Control_Totals_3e_{date_value}",
                    output_dir=notebook_dir,
                    delimiter=','
                )
                csv_3e_path = csv_3e_paths[0] if csv_3e_paths else None
                if csv_3e_path:
                    ux.info(f"  Raw data saved to: {csv_3e_path.name}")
            
        except Exception as e:
            ux.error(f"✗ Error executing 3e script: {str(e)}")
            step.log(f"Error executing 3e script: {str(e)}")

### Validate GeoHaz Thresholds

In [ ]:
# Validate geocoding results against configuration thresholds
validation_results = None
geohaz_all_passed = False
geohaz_summary = {}

if execution_failed:
    ux.warning("⏭ Skipping GeoHaz validation due to previous error")
elif geocoding_results is None or geocoding_results.empty:
    ux.warning("⚠ No geocoding results to validate")
else:
    ux.subheader("GeoHaz Threshold Validation")
    
    # Get GeoHaz thresholds from configuration
    geohaz_thresholds = config_data.get('GeoHaz Thresholds', [])
    
    if not geohaz_thresholds:
        ux.warning("⚠ No GeoHaz thresholds found in configuration - skipping validation")
    else:
        # Get portfolio to import file mapping from configuration
        import_file_mapping = get_import_file_mapping_from_config(config_data)
        
        # Perform validation
        validation_results, geohaz_all_passed = validate_geohaz_thresholds(
            geocoding_results=geocoding_results,
            geohaz_thresholds=geohaz_thresholds,
            import_file_mapping=import_file_mapping
        )
        
        # Handle empty validation results (e.g., no matching portfolios or all portfolios empty)
        if validation_results is None or validation_results.empty:
            ux.warning("⚠ No geocoding results matched the configured thresholds")
            ux.info("This may occur when:")
            ux.info("  - No portfolios in geocoding results match Import File names in thresholds")
            ux.info("  - All matching portfolios had zero locations")
            step.log("GeoHaz validation: No matching results to validate")
        else:
            # Display summary
            total_checks = len(validation_results)
            passed_checks = len(validation_results[validation_results['Status'] == 'PASS'])
            failed_checks = len(validation_results[validation_results['Status'] == 'FAIL'])
            
            geohaz_summary = {
                'total_checks': int(total_checks),
                'passed_checks': int(passed_checks),
                'failed_checks': int(failed_checks),
                'all_passed': bool(geohaz_all_passed)
            }
            
            if geohaz_all_passed:
                ux.success(f"✓ All {total_checks} geocoding thresholds met")
            else:
                ux.warning(f"⚠ {failed_checks} of {total_checks} geocoding thresholds not met")
            
            # Log validation status
            if geohaz_all_passed:
                step.log(f"GeoHaz validation: All {total_checks} thresholds met")
            else:
                step.log(f"GeoHaz validation: {failed_checks}/{total_checks} thresholds failed")

### Export GeoHaz Validation to Excel

In [ ]:
# Export validation results to Excel (one sheet per Import File)
excel_geohaz_path = None

if validation_results is not None and not validation_results.empty:
    # Get the notebook directory for output
    notebook_dir = context.notebook_path.parent
    
    excel_geohaz_path = save_geohaz_validation_to_excel(
        validation_results=validation_results,
        date_value=date_value,
        cycle_type=cycle_type,
        output_dir=notebook_dir
    )
    
    if excel_geohaz_path:
        ux.success(f"✓ GeoHaz validation exported to: {excel_geohaz_path.name}")
        step.log(f"GeoHaz validation exported to: {excel_geohaz_path}")
else:
    ux.info("No GeoHaz validation results to export")

## 8) Compare RMS EDM (3d) vs Geocoding Summary (3e) Control Totals

Compares control totals between 3d (RMS EDM) and 3e (Geocoding Summary) for base portfolios only.

**Attributes compared:**
- RiskCount (3e) vs LocationCountDistinct (3d)
- TIV (3e) vs LocationLimit (3d)
- TRV (3e) vs TotalReplacementValue (3d)

**A difference of 0 means the values match.**

In [ ]:
# Compare 3d vs 3e control totals
comparison_3d_vs_3e = None
all_matched_3d_vs_3e = False
summary_3d_vs_3e = {}

if execution_failed:
    ux.warning("Skipping 3d vs 3e comparison due to previous error")
elif not edm_results or geocoding_results is None or geocoding_results.empty:
    ux.warning("Cannot compare: Missing 3d or 3e results")
else:
    ux.subheader("3d vs 3e Comparison Results")

    try:
        # Get base portfolio names from configuration (using existing function)
        base_portfolio_list = get_base_portfolios(config_data.get('Portfolios', []))
        base_portfolio_names = [p['Portfolio'] for p in base_portfolio_list]
        ux.info(f"Base portfolios: {', '.join(base_portfolio_names)}")

        # Run comparison (pivot format)
        comparison_3d_vs_3e, all_matched_3d_vs_3e = compare_3d_vs_3e_pivot(
            edm_results,
            [geocoding_results],
            base_portfolio_names
        )

        if comparison_3d_vs_3e is not None and not comparison_3d_vs_3e.empty:
            # Split into Flood and Non-Flood
            is_flood = comparison_3d_vs_3e['PORTNAME'].str.startswith('USFL_')
            flood_results_3d_3e = comparison_3d_vs_3e[is_flood]
            non_flood_results_3d_3e = comparison_3d_vs_3e[~is_flood]

            # Display summary
            total = len(comparison_3d_vs_3e)
            matched = (comparison_3d_vs_3e['Status'] == 'MATCH').sum()

            summary_3d_vs_3e = {
                'total': total,
                'matched': int(matched),
                'mismatched': total - int(matched),
                'all_matched': bool(all_matched_3d_vs_3e)
            }

            if all_matched_3d_vs_3e:
                ux.success(f"All {total} base portfolios match")
            else:
                ux.warning(f"{total - matched} of {total} base portfolios have mismatches")

            step.log(f"3d vs 3e comparison: {matched}/{total} matched")
        else:
            ux.warning("No comparison results generated")

    except Exception as e:
        ux.error(f"Error comparing 3d vs 3e: {str(e)}")
        step.log(f"Error comparing 3d vs 3e: {str(e)}")

### Export Control Totals to Excel

Exports both 3b vs 3d and 3d vs 3e comparison results to a single Excel file with three sheets:
- **3b_vs_3d_NonFlood**: Non-Flood perils comparison
- **3b_vs_3d_Flood**: Flood perils comparison  
- **3d_vs_3e**: Base portfolios comparison (all perils combined)

In [ ]:
# Export comparison results to Excel (combined 3b vs 3d and 3d vs 3e)
excel_control_totals_path = None

# Check if we have any results to export
has_3b_vs_3d = comparison_results is not None and not comparison_results.empty
has_3d_vs_3e = comparison_3d_vs_3e is not None and not comparison_3d_vs_3e.empty

if has_3b_vs_3d or has_3d_vs_3e:
    notebook_dir = context.notebook_path.parent

    excel_control_totals_path = save_control_totals_3b_vs_3d_to_excel(
        comparison_results_3b_vs_3d=comparison_results,
        date_value=date_value,
        cycle_type=cycle_type,
        output_dir=notebook_dir,
        comparison_results_3d_vs_3e=comparison_3d_vs_3e
    )

    if excel_control_totals_path:
        sheets_written = []
        if has_3b_vs_3d:
            sheets_written.extend(['3b_vs_3d_NonFlood', '3b_vs_3d_Flood'])
        if has_3d_vs_3e:
            sheets_written.append('3d_vs_3e')
        ux.success(f"Control totals exported to: {excel_control_totals_path.name}")
        ux.info(f"  Sheets: {', '.join(sheets_written)}")
        step.log(f"Control totals exported to: {excel_control_totals_path}")
else:
    ux.info("No comparison results to export")

## 9) Complete Step Execution

In [ ]:
# Complete step execution
ux.header("Step Completion")

if execution_failed:
    # Handle configuration/execution failure
    from helpers.step import update_step_run
    from helpers.constants import StepStatus
    
    update_step_run(step.run_id, StepStatus.FAILED, error_message=error_message)
    
    ux.error("\n" + "="*60)
    ux.error("CONTROL TOTALS EXECUTION FAILED")
    ux.error("="*60)
    ux.error(f"\nError: {error_message}")
    ux.info("\nPlease fix the error and retry.")

else:
    # Build comparison summary for output
    comparison_summary = {}
    if comparison_results is not None and not comparison_results.empty:
        total_groups = len(comparison_results)
        matched_groups = int((comparison_results['Status'] == 'MATCH').sum())
        comparison_summary = {
            'total_exposure_groups': total_groups,
            'matched_groups': matched_groups,
            'mismatched_groups': total_groups - matched_groups,
            'all_matched': bool(all_matched),
            'non_flood': non_flood_summary,
            'flood': flood_summary
        }
    
    # Complete the step successfully
    output_data = {
        'date_value': date_value,
        'cycle_type': cycle_type,
        'import_file_result_count': len(import_file_results),
        'edm_result_count': len(edm_results),
        '3b_vs_3d_comparison': comparison_summary,
        '3d_vs_3e_comparison': summary_3d_vs_3e,
        'geohaz_validation': geohaz_summary
    }
    step.complete(output_data)

    ux.success("\n" + "="*60)
    ux.success("✓ CONTROL TOTALS VALIDATION COMPLETED")
    ux.success("="*60)
    
    # Display 3b vs 3d summary
    ux.info(f"\nImport File (3b): {len(import_file_results)} result set(s)")
    ux.info(f"RMS EDM (3d): {len(edm_results)} result set(s)")
    
    if comparison_summary:
        ux.info(f"\n3b vs 3d Comparison:")
        
        # Non-Flood summary
        if non_flood_summary:
            if non_flood_summary['mismatched'] == 0:
                ux.success(f"  Non-Flood: ✓ All {non_flood_summary['total']} groups match")
            else:
                ux.warning(f"  Non-Flood: ⚠ {non_flood_summary['mismatched']}/{non_flood_summary['total']} groups have mismatches")
        
        # Flood summary
        if flood_summary:
            if flood_summary['mismatched'] == 0:
                ux.success(f"  Flood: ✓ All {flood_summary['total']} groups match")
            else:
                ux.warning(f"  Flood: ⚠ {flood_summary['mismatched']}/{flood_summary['total']} groups have mismatches")
    
    # Display 3d vs 3e summary
    if summary_3d_vs_3e:
        ux.info(f"\n3d vs 3e Comparison (Base Portfolios):")
        if summary_3d_vs_3e.get('all_matched'):
            ux.success(f"  ✓ All {summary_3d_vs_3e['total']} base portfolios match")
        else:
            ux.warning(f"  ⚠ {summary_3d_vs_3e['mismatched']}/{summary_3d_vs_3e['total']} base portfolios have mismatches")
    
    # Display GeoHaz summary
    if geohaz_summary:
        ux.info(f"\nGeoHaz Validation: {geohaz_summary['passed_checks']}/{geohaz_summary['total_checks']} checks passed")
        if geohaz_summary['all_passed']:
            ux.success("  ✓ All geocoding thresholds met")
        else:
            ux.warning(f"  ⚠ {geohaz_summary['failed_checks']} threshold(s) not met")
    elif validation_results is not None and validation_results.empty:
        ux.info("\nGeoHaz validation: No matching results to validate")
    else:
        ux.info("\nGeoHaz validation: Skipped (no thresholds configured or no results)")
    
    # Display exported files
    ux.info("\nExported Files:")
    if csv_3b_path:
        ux.info(f"  - {csv_3b_path.name}")
    if csv_3d_path:
        ux.info(f"  - {csv_3d_path.name}")
    if csv_3e_path:
        ux.info(f"  - {csv_3e_path.name}")
    if excel_control_totals_path:
        ux.info(f"  - {excel_control_totals_path.name}")
    if excel_geohaz_path:
        ux.info(f"  - {excel_geohaz_path.name}")
    
    ux.info("\nNext: Proceed to Stage 04 (Analysis Execution)")

    # Send Teams notification for milestone completion
    try:
        from helpers.teams_notification import TeamsNotificationClient, build_notification_actions
        from helpers.database import get_current_schema

        teams = TeamsNotificationClient()
        schema = get_current_schema()
        actions = build_notification_actions(
            notebook_path=str(context.notebook_path),
            cycle_name=context.cycle_name,
            schema=schema
        )
        
        # Build comparison status message
        comparison_msg = ""
        if comparison_summary:
            if non_flood_summary:
                if non_flood_summary['mismatched'] == 0:
                    comparison_msg += f"- Non-Flood: ✓ All {non_flood_summary['total']} groups match\n"
                else:
                    comparison_msg += f"- Non-Flood: ⚠ {non_flood_summary['mismatched']}/{non_flood_summary['total']} groups have mismatches\n"
            if flood_summary:
                if flood_summary['mismatched'] == 0:
                    comparison_msg += f"- Flood: ✓ All {flood_summary['total']} groups match\n"
                else:
                    comparison_msg += f"- Flood: ⚠ {flood_summary['mismatched']}/{flood_summary['total']} groups have mismatches\n"
        
        # Build 3d vs 3e status message
        comparison_3d_3e_msg = ""
        if summary_3d_vs_3e:
            if summary_3d_vs_3e.get('all_matched'):
                comparison_3d_3e_msg = f"- 3d vs 3e: ✓ All {summary_3d_vs_3e['total']} base portfolios match\n"
            else:
                comparison_3d_3e_msg = f"- 3d vs 3e: ⚠ {summary_3d_vs_3e['mismatched']}/{summary_3d_vs_3e['total']} base portfolios have mismatches\n"
        
        # Build GeoHaz status message
        geohaz_msg = ""
        if geohaz_summary:
            if geohaz_summary['all_passed']:
                geohaz_msg = f"- GeoHaz: ✓ All {geohaz_summary['total_checks']} thresholds met\n"
            else:
                geohaz_msg = f"- GeoHaz: ⚠ {geohaz_summary['failed_checks']}/{geohaz_summary['total_checks']} thresholds not met\n"

        teams.send_success(
            title=f"[{context.cycle_name}] Stage 03 Control Totals Complete",
            message=f"**Cycle:** {context.cycle_name}\n"
                    f"**Stage:** {context.stage_name}\n"
                    f"**Step:** {context.step_name}\n\n"
                    f"**Results:**\n"
                    f"- Import File (3b): {len(import_file_results)} result set(s)\n"
                    f"- RMS EDM (3d): {len(edm_results)} result set(s)\n\n"
                    f"**3b vs 3d Comparison:**\n"
                    f"{comparison_msg}\n"
                    f"**3d vs 3e Comparison:**\n"
                    f"{comparison_3d_3e_msg}\n"
                    f"**GeoHaz Validation:**\n"
                    f"{geohaz_msg}\n"
                    f"Ready to proceed to Stage 04 Analysis Execution.",
            actions=actions
        )
        ux.info("\nTeams notification sent.")
    except Exception as e:
        ux.warning(f"\nCould not send Teams notification: {str(e)}")